In [16]:
import numpy as np
import pandas as pd
from scipy import stats

%precision 3
np.random.seed(1111)
np.set_printoptions(precision=3)

# 統計的仮説検定

In [2]:
# 標本 Data: 14日間のフライドポテトの重さを準備
df = pd.read_csv('../data/ch11_potato.csv')
sample = np.array(df['重さ'])
display(sample)

array([122.02, 131.73, 130.6 , 131.82, 132.05, 126.12, 124.43, 132.89,
       122.79, 129.95, 126.14, 134.45, 127.64, 125.68])

In [3]:
# 標本平均を求める
s_mean = np.mean(sample)
s_mean

128.451

## 統計的仮説検定とは（statistical hypothesis testing）

### 統計的仮説検定の基本
- フライドポテトの母平均が130gより少ないかどうかを確かめたい。
- 前提として
  - 母集団が正規分布に従っている
  - 母分散は９
とわかっている。

母平均が「130g」という仮定をすると、標本平均は、N(130, 9)に従うことになる

In [4]:
# 標本平均が P(標本平均<=x)=0.05 を満たす x を考える
rv = stats.norm(130, np.sqrt(9 / 14))
rv.isf(0.95)

128.681

標本平均が 126.681g 以下の重さになることは５％の確率でしか発生しない = 標本平均が 128.451g となったのは５％の確率でしか発生しない珍事な出来事といえる。
よって「母平均が 130g より少ない」と考え結論づける。

In [5]:
# 帰無仮説「母平均 130g」
# 検定統計量に標本平均を標準化した Z を使用する。
# 検定統計量が臨界値より小さければ帰無仮説を棄却し、そうでないときに帰無仮説を採択する。

z = (s_mean - 130) / np.sqrt(9/14)
print(f'統計検定量: {z:.3f}')

統計検定量: -1.932


In [6]:
rv = stats.norm()

print(f'臨界値: {rv.isf(0.95):.3f}')

臨界値: -1.645


検定統計量と臨界値を比較すると検定統計量のほうが小さい値となっている。
= 帰無仮説は棄却
= 平均は、130g より少ないという結論になる

In [7]:
# ｐ値を使った仮説検定について確認する。

# 検定統計量（標本平均を標準化した Z）からｐ値を求める
print(f'ｐ値: {rv.cdf(z):.3f}')

ｐ値: 0.027


ｐ値が有意水準 0.05 より小さい値になった。
= 帰無仮説は棄却

### 片側検定と両側検定
フライドポテトの「母平均は 130g ではない」という対立仮説で仮説検定を行なう。（※130g より少ない、多いの両側検定）

In [10]:
# 検定統計量を求める
z = (s_mean - 130) / np.sqrt(9/14)
print(f'検定統計量: {z:.3f}（※標本平均を標準化した値）')

検定統計量: -1.932（※標本平均を標準化した値）


In [19]:
# 臨界値を求める
# 両側検定のため標準正規分布の95%区間によって求める
rv = stats.norm()
rv.interval(0.95)

(-1.960, 1.960)

臨界値と統計検定量を比較してみると、統計検定量が採択域にはいっているため帰無仮説は棄却されない。
※両側検定に比べると片側検定は棄却しやすい。

In [21]:
# ｐ値を求める
# 両側検定のｐ値は、上側と下側の両方の面積を考慮し、累積密度関数の値を２倍にすることによって求められる。
print(f'ｐ値（両側検定: {rv.cdf(z) * 2:.3f}')

ｐ値（両側検定: 0.053


ｐ値が有意水準 0.05 より大きくなっていることからも帰無仮説は棄却されない。